In [19]:
import Interface 
import os 
from dotenv import load_dotenv

load_dotenv()

True

In [20]:
Interface.create_db(dbname=os.getenv("DATABASE_NAME"))

Database 'dds_assgn1' already exists


In [21]:
conn = None 
try:
    conn = Interface.getopenconnection(dbname=os.getenv("DATABASE_NAME"))
    Interface.roundrobinpartition(
        'ratings',
        10,
        conn 
    )
except Exception as e:
    print(e)
    raise 
finally:
    conn.close()
    

syntax error at or near ","
LINE 3:         SELECT , , ROW_NUMBER() OVER () AS rnum
                       ^

syntax error at or near ","
LINE 3:         SELECT , , ROW_NUMBER() OVER () AS rnum
                       ^



SyntaxError: syntax error at or near ","
LINE 3:         SELECT , , ROW_NUMBER() OVER () AS rnum
                       ^


In [35]:
# conn = Interface.getopenconnection(dbname='dds_assgn1')
# Interface.loadratings('ratings', './ml-10M100K/ratings.dat', conn)
# conn.close()

In [38]:
conn = Interface.getopenconnection(dbname=os.getenv("DATABASE_NAME"))
Interface.rangepartition('ratings', 5, conn)
conn.close()

In [36]:
def drop_range_part_tables(n):
    conn = None
    try:
        conn = Interface.getopenconnection(dbname=os.getenv("DATABASE_NAME"))
        Interface.rangepartition('ratings', n, conn) 
        cur = conn.cursor()

        # Tìm tất cả các bảng có tên bắt đầu với 'range_part'
        cur.execute("""
            SELECT tablename 
            FROM pg_tables 
            WHERE tablename LIKE 'range_part%%' AND schemaname = 'public'
        """)
        tables = cur.fetchall()

        for table in tables:
            table_name = table[0]
            
            # Đếm số dòng
            cur.execute(f'SELECT COUNT(*) FROM "{table_name}"')
            row_count = cur.fetchone()[0]
            print(row_count, end=" ")

            # Xóa bảng
            cur.execute(f'DROP TABLE IF EXISTS "{table_name}" CASCADE')
        print()

        conn.commit()
        cur.close()
    except Exception as e:
        print(f"Error: {e}")
    finally:
        if conn:
            conn.close()

In [37]:
for i in range(5, 20):
    drop_range_part_tables(i)

479168 908584 2726854 3755614 2129834 
94988 502458 1160484 2356676 3755614 2129834 
94988 384180 908584 370178 3236440 2875850 2129834 
94988 384180 118278 1160484 2356676 879764 2875850 2129834 
94988 384180 118278 790306 370178 2356676 879764 2875850 2129834 
94988 384180 118278 790306 370178 2356676 879764 2875850 585022 1544812 
0 94988 0 384180 118278 790306 370178 2356676 879764 2875850 585022 
0 94988 585022 1544812 384180 118278 790306 370178 0 2356676 879764 2875850 
0 94988 2875850 585022 1544812 384180 118278 0 790306 370178 2356676 0 879764 
0 94988 0 2875850 585022 1544812 384180 0 118278 790306 370178 0 2356676 879764 
0 94988 384180 0 118278 0 0 370178 2356676 0 879764 2875850 0 585022 0 
0 94988 0 384180 118278 0 790306 370178 0 2356676 0 879764 2875850 0 585022 1544812 
0 94988 0 384180 0 118278 790306 0 370178 0 2356676 879764 0 2875850 0 585022 1544812 
0 94988 0 384180 0 118278 0 790306 370178 0 2356676 0 879764 0 2875850 0 585022 1544812 
0 94988 0 384180 0 118278